In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 12.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whee

In [3]:
import os

DIR = "/content/drive/MyDrive/Competitions/probspace/研究論文の国際学会採択予測"
INPUT_DIR = os.path.join(DIR,"input")
OUTPUT_DIR = os.path.join(DIR,"output")

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [4]:
class CFG:
  debug=False
  ver=7
  seed=42
  model="lightgbm"
  n_folds = 15
  target_col="y"
  n_component=128

In [5]:
import pandas as pd
import numpy as np

train = pd.read_csv(os.path.join(INPUT_DIR,"train_data.csv"))
test = pd.read_csv(os.path.join(INPUT_DIR,"test_data.csv"))
sample_sub = pd.read_csv(os.path.join(INPUT_DIR,"submission.csv"))

train["src"]="train"
test["src"]="test"

df = pd.concat([train,test],ignore_index=True)

print(train.shape)
display(train.head(3))

print(test.shape)
display(test.head(3))

print(sample_sub.shape)
display(sample_sub.head(3))

(4974, 7)


,id,title,year,abstract,keywords,y,src
0,1,Hierarchical Adversarially Learned Inference,2018,We propose a novel hierarchical generative mod...,"generative, hierarchical, unsupervised, semisu...",0,train
1,2,Learning to Compute Word Embeddings On the Fly,2018,Words in natural language follow a Zipfian dis...,"NLU, word embeddings, representation learning",0,train
2,3,Graph2Seq: Scalable Learning Dynamics for Graphs,2018,Neural networks are increasingly used as a gen...,NaN,0,train


(6393, 6)


,id,title,year,abstract,keywords,src
0,1,StyleAlign: Analysis and Applications of Align...,2022,"In this paper, we perform an in-depth study of...","StyleGAN, transfer learning, fine tuning, mode...",test
1,2,Embedding a random graph via GNN: mean-field i...,2021,We develop a theory for embedding a random gra...,"Graph neural network, graph embedding, multi-r...",test
2,3,BBRefinement: an universal scheme to improve p...,2021,We present a conceptually simple yet powerful ...,"object detection, deep neural networks, refine...",test


(6393, 2)


,id,y
0,1,0
1,2,0
2,3,0


In [6]:
train.isnull().sum()

id            0
title         0
year          0
abstract      0
keywords    480
y             0
src           0
dtype: int64

In [7]:
test.isnull().sum()

id            0
title         0
year          0
abstract      0
keywords    775
src           0
dtype: int64

In [8]:
train["num_title_word"] = train["title"].apply(lambda x: len(x.split()))
train["num_abst_word"] = train["abstract"].apply(lambda x: len(x.split()))

test["num_title_word"] = test["title"].apply(lambda x: len(x.split()))
test["num_abst_word"] = test["abstract"].apply(lambda x: len(x.split()))

In [9]:
#train["full_text"] = train["title"] + "[SEP]" + train["abstract"]
#test["full_text"] = test["title"] + "[SEP]" + test["abstract"]

train["full_text"] = train["abstract"]
test["full_text"] = test["abstract"]

In [10]:
if CFG.debug:
  print(train.shape)
  print(test.shape)
  train = train.sample(n=500, random_state=CFG.seed).reset_index(drop=True)
  test = test.sample(n=500, random_state=CFG.seed).reset_index(drop=True)
  print(train.shape)
  print(test.shape)

In [11]:
from transformers import AutoModel,AutoTokenizer
import torch
import torch.nn.functional as F
from tqdm import tqdm
import gc
import random

In [12]:
def seed_everything(seed=CFG.seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=CFG.seed)

In [13]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state.detach().cpu()
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

In [14]:
BATCH_SIZE = 3

class EmbedDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.df = df.reset_index(drop=True)
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        text = self.df.loc[idx,"full_text"]
        tokens = tokenizer(
                text,
                None,
                add_special_tokens=True,
                padding='max_length',
                truncation=True,
                max_length=MAX_LEN,return_tensors="pt")
        tokens = {k:v.squeeze(0) for k,v in tokens.items()}
        return tokens

ds_tr = EmbedDataset(train)
embed_dataloader_tr = torch.utils.data.DataLoader(ds_tr,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False)
ds_te = EmbedDataset(test)
embed_dataloader_te = torch.utils.data.DataLoader(ds_te,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False)

In [15]:
tokenizer = None
MAX_LEN = 640

def get_embeddings(MODEL_NM='', MAX=640, BATCH_SIZE=3, verbose=True):
    global tokenizer, MAX_LEN
    DEVICE="cuda"
    model = AutoModel.from_pretrained( MODEL_NM )
    tokenizer = AutoTokenizer.from_pretrained( MODEL_NM )
    MAX_LEN = MAX
    
    model = model.to(DEVICE)
    model.eval()
        
    all_train_text_feats = []
    for batch in tqdm(embed_dataloader_tr,total=len(embed_dataloader_tr)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        all_train_text_feats.extend(sentence_embeddings)
    all_train_text_feats = np.array(all_train_text_feats)
    if verbose:
        print('Train embeddings shape',all_train_text_feats.shape)

    te_text_feats = []
    for batch in tqdm(embed_dataloader_te,total=len(embed_dataloader_te)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        te_text_feats.extend(sentence_embeddings)
    te_text_feats = np.array(te_text_feats)
    if verbose:
        print('Test embeddings shape',te_text_feats.shape)
        
    return all_train_text_feats, te_text_feats

In [16]:
MODEL_NM = 'microsoft/deberta-base'
all_train_text_feats, te_text_feats = get_embeddings(MODEL_NM)

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


100%|██████████| 1658/1658 [01:37<00:00, 16.99it/s]


Train embeddings shape (4974, 768)


100%|██████████| 2131/2131 [02:02<00:00, 17.39it/s]

Test embeddings shape (6393, 768)


In [17]:
MODEL_NM = 'microsoft/deberta-v3-large'
all_train_text_feats2, te_text_feats2 = get_embeddings(MODEL_NM)

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 1658/1658 [04:14<00:00,  6.50it/s]


Train embeddings shape (4974, 1024)


100%|██████████| 2131/2131 [05:28<00:00,  6.49it/s]

Test embeddings shape (6393, 1024)


In [18]:
MODEL_NM = 'microsoft/deberta-large'
all_train_text_feats3, te_text_feats3 = get_embeddings(MODEL_NM)

Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


100%|██████████| 1658/1658 [04:18<00:00,  6.40it/s]


Train embeddings shape (4974, 1024)


100%|██████████| 2131/2131 [05:33<00:00,  6.39it/s]

Test embeddings shape (6393, 1024)


In [19]:
#MODEL_NM = 'microsoft/deberta-large-mnli'
#all_train_text_feats4, te_text_feats4 = get_embeddings(MODEL_NM, MAX=512)

In [20]:
MODEL_NM = 'microsoft/deberta-xlarge'
all_train_text_feats5, te_text_feats5 = get_embeddings(MODEL_NM, MAX=512)

Some weights of the model checkpoint at microsoft/deberta-xlarge were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


100%|██████████| 1658/1658 [06:35<00:00,  4.20it/s]


Train embeddings shape (4974, 1024)


100%|██████████| 2131/2131 [08:28<00:00,  4.19it/s]

Test embeddings shape (6393, 1024)


In [21]:
#MODEL_NM = 'microsoft/deberta-v2-xlarge'
#all_train_text_feats6, te_text_feats6 = get_embeddings(MODEL_NM, MAX=512)

In [22]:
MODEL_NM = 'microsoft/deberta-v3-base'
all_train_text_feats7, te_text_feats7 = get_embeddings(MODEL_NM)

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 1658/1658 [01:34<00:00, 17.54it/s]


Train embeddings shape (4974, 768)


100%|██████████| 2131/2131 [02:01<00:00, 17.49it/s]

Test embeddings shape (6393, 768)


In [23]:
all_train_text_feats = np.concatenate([all_train_text_feats,all_train_text_feats2,
                                       all_train_text_feats3,#all_train_text_feats4,
                                       all_train_text_feats5, #all_train_text_feats6,
                                     all_train_text_feats7],axis=1)

te_text_feats = np.concatenate([te_text_feats,te_text_feats2,
                                te_text_feats3,#te_text_feats4,
                                te_text_feats5,#te_text_feats6,
                                te_text_feats7],axis=1)

del all_train_text_feats2, te_text_feats2
del all_train_text_feats3, te_text_feats3
#del all_train_text_feats4, te_text_feats4
del all_train_text_feats5, te_text_feats5
#del all_train_text_feats6, te_text_feats6
del all_train_text_feats7, te_text_feats7
gc.collect()

print('Our concatenated embeddings have shape', all_train_text_feats.shape )

Our concatenated embeddings have shape (4974, 4608)


In [24]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=CFG.n_component)
all_train_text_feats = svd.fit_transform(all_train_text_feats)
te_text_feats = svd.transform(te_text_feats)
print('Our concatenated embeddings have shape', all_train_text_feats.shape)

Our concatenated embeddings have shape (4974, 128)


In [25]:
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
skf = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
for i,(train_index, val_index) in enumerate(skf.split(train,train[CFG.target_col])):
    train.loc[val_index,'fold'] = i

print('Train samples per fold:')
train["fold"] = train["fold"].astype(int)
display(train.groupby("fold").size())

Train samples per fold:


fold
0     332
1     332
2     332
3     332
4     332
5     332
6     332
7     332
8     332
9     331
10    331
11    331
12    331
13    331
14    331
dtype: int64

In [26]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

params = {
        'objective': "binary",
        'metric': 'binary_logloss',
        'learning_rate': 0.05,
        'boosting': "gbdt",
        'seed': CFG.seed,
        "reg_alpha":0.1,
        "n_estimators":1000
    }

oof_df = pd.DataFrame()
scores = []
preds = []
for fold in range(CFG.n_folds):
    print('#'*25)
    print('### Fold',fold+1)
    print('#'*25)
    
    tr_ = train[train["fold"]!=fold]
    ev_ = train[train["fold"]==fold]
    
    tr_feats = all_train_text_feats[list(tr_.index)]
    ev_feats = all_train_text_feats[list(ev_.index)]

    clf = lgb.LGBMClassifier(**params)
    clf.fit(tr_feats, tr_[CFG.target_col].values,
              eval_set=[(ev_feats, ev_[CFG.target_col].values)],
              verbose = 200,
              early_stopping_rounds=50)
    ev_preds = clf.predict_proba(ev_feats)
    ev_["pred_values"] = ev_preds[:,1]
    oof_df = pd.concat([oof_df, ev_])

    test_pred = clf.predict_proba(te_text_feats)
    print()
    score = accuracy_score(ev_[CFG.target_col].values, (ev_["pred_values"]>.5).astype(int))
    scores.append(score)
    print("Fold : {} Accuracy score: {}".format(fold,score))
    preds.append(test_pred[:,1])
    
print('#'*25)
print('Overall CV Accuracy =',np.mean(scores))
print('OOF CV Accuracy = ',accuracy_score(oof_df[CFG.target_col].values, (oof_df["pred_values"]>.5).astype(int)))

#########################
### Fold 1
#########################
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[75]	valid_0's binary_logloss: 0.57902

Fold : 0 Accuracy score: 0.7108433734939759
#########################
### Fold 2
#########################
Training until validation scores don't improve for 50 rounds.


<ipython-input-26-fe25836c593b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev_["pred_values"] = ev_preds[:,1]


Early stopping, best iteration is:
[81]	valid_0's binary_logloss: 0.575565

Fold : 1 Accuracy score: 0.7018072289156626
#########################
### Fold 3
#########################
Training until validation scores don't improve for 50 rounds.


<ipython-input-26-fe25836c593b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev_["pred_values"] = ev_preds[:,1]


Early stopping, best iteration is:
[80]	valid_0's binary_logloss: 0.575254

Fold : 2 Accuracy score: 0.7048192771084337
#########################
### Fold 4
#########################
Training until validation scores don't improve for 50 rounds.


...

<ipython-input-26-fe25836c593b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev_["pred_values"] = ev_preds[:,1]


[200]	valid_0's binary_logloss: 0.565519
Early stopping, best iteration is:
[153]	valid_0's binary_logloss: 0.562642

Fold : 13 Accuracy score: 0.716012084592145
#########################
### Fold 15
#########################
Training until validation scores don't improve for 50 rounds.


<ipython-input-26-fe25836c593b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev_["pred_values"] = ev_preds[:,1]


Early stopping, best iteration is:
[69]	valid_0's binary_logloss: 0.586377

Fold : 14 Accuracy score: 0.7069486404833837
#########################
Overall CV Accuracy = 0.6994388429852341
OOF CV Accuracy =  0.6994370727784479


<ipython-input-26-fe25836c593b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev_["pred_values"] = ev_preds[:,1]


In [27]:
best_score = 0
best_thresh = 0.5
for thresh in np.arange(0.1, 0.70, 0.01):
    thresh = np.round(thresh, 2)
    score = accuracy_score(oof_df[CFG.target_col], (oof_df["pred_values"]>thresh).astype(int))
    print("Accuracy score at threshold {0} is {1}".format(thresh, score))
    if score > best_score:
      best_score = score
      best_thresh = thresh
print()
print("best Accuracy score at threshold {0} is {1}".format(best_thresh, accuracy_score(oof_df[CFG.target_col], (oof_df.pred_values>best_thresh).astype(int))))

Accuracy score at threshold 0.1 is 0.3357458785685565
Accuracy score at threshold 0.11 is 0.34439083232810613
Accuracy score at threshold 0.12 is 0.3538399678327302
Accuracy score at threshold 0.13 is 0.3657016485725774
Accuracy score at threshold 0.14 is 0.37997587454764775
Accuracy score at threshold 0.15 is 0.3936469642139123
Accuracy score at threshold 0.16 is 0.40952955367913146
Accuracy score at threshold 0.17 is 0.4223964616003217
Accuracy score at threshold 0.18 is 0.43687173301166066
Accuracy score at threshold 0.19 is 0.45456373140329714
Accuracy score at threshold 0.2 is 0.4734620024125452
Accuracy score at threshold 0.21 is 0.4881383192601528
Accuracy score at threshold 0.22 is 0.5058303176517893
Accuracy score at threshold 0.23 is 0.5211097708082026
Accuracy score at threshold 0.24 is 0.5319662243667069
Accuracy score at threshold 0.25 is 0.5486529955770004
Accuracy score at threshold 0.26 is 0.5657418576598311
Accuracy score at threshold 0.27 is 0.5800160836349015
Accurac

In [28]:
test_pred = np.mean(preds,axis=0)
sub = test.copy()
sub[CFG.target_col] = test_pred
sub[CFG.target_col] = (sub[CFG.target_col]>best_thresh).astype(int)


sub[["id","y"]].to_csv(os.path.join(OUTPUT_DIR, f'submit_{CFG.model}_seed{CFG.seed}_ver{CFG.ver}.csv'),index=False)
display(sub[["id","y"]])
display(sub.y.value_counts())

,id,y
0,1,1
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
6388,6389,0
6389,6390,0
6390,6391,0
6391,6392,0


0    5922
1     471
Name: y, dtype: int64

In [ ]:
from google.colab import runtime
runtime.unassign()